<a href="https://colab.research.google.com/github/juveriya-Anjum/brain-tumor-detection/blob/main/Tumor_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  built a model of Vision Transformers for Tumor Detection


Detects brain tumors in medical images using Vision Transformers.

In [ ]:
!pip install transformers timm torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")
model.classifier = torch.nn.Linear(in_features=768, out_features=2)  # Set to 2 classes


In [ ]:
for param in model.vit.parameters():  # Freeze ViT backbone
    param.requires_grad = False


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Defines Loss Function
criterion = nn.CrossEntropyLoss()

# Defines Optimizer
optimizer = optim.AdamW(model.parameters(), lr=2e-5, weight_decay=1e-4)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),  # Faster training
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])


In [ ]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm  # For progress bar

# Define Dataset Class for "Yes" (Tumor) and "No" (No Tumor)
class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Class mapping: 1 for "yes" (Tumor), 0 for "no" (No Tumor)
        self.class_map = {"yes": 1, "no": 0}

        for label in self.class_map.keys():
            class_path = os.path.join(root_dir, label)
            if not os.path.exists(class_path):
                print(f"Warning: Folder '{class_path}' not found. Check dataset path!")
                continue  # Skip missing folders
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                self.image_paths.append(img_path)
                self.labels.append(self.class_map[label])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # Load Image
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (224, 224))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Apply Transformations
        if self.transform:
            image = self.transform(image)

        return image, label

# Set Dataset Path
DATASET_PATH = "/content/drive/MyDrive/DL_CS/brain_tumor_dataset"  # Change this to your dataset location

# Define Transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load Dataset
dataset = BrainTumorDataset(DATASET_PATH, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Verify Dataset is Loaded
print(f"Total images loaded: {len(dataset)}")
if len(dataset) == 0:
    print(" Error: No images found. Check the dataset path.")

# Training Function
def train(model, dataloader, criterion, optimizer, epochs=3):
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0

        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images).logits
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            loop.set_postfix(loss=running_loss / len(dataloader))

        print(f"Epoch {epoch+1} Loss: {running_loss / len(dataloader)}")

# Start Training (Adjust epochs as needed)
train(model, dataloader, criterion, optimizer, epochs=5)


✅ Total images loaded: 253


Epoch 1/5: 100%|██████████| 8/8 [03:15<00:00, 24.47s/it, loss=0.638]


Epoch 1 Loss: 0.6378135085105896


Epoch 2/5: 100%|██████████| 8/8 [02:36<00:00, 19.55s/it, loss=0.647]


Epoch 2 Loss: 0.6466182917356491


...

Epoch 3 Loss: 0.6374817565083504


Epoch 4/5: 100%|██████████| 8/8 [02:35<00:00, 19.46s/it, loss=0.64]


Epoch 4 Loss: 0.639967754483223


Epoch 5/5: 100%|██████████| 8/8 [02:36<00:00, 19.56s/it, loss=0.631]

Epoch 5 Loss: 0.6305329725146294


In [ ]:
from sklearn.model_selection import train_test_split

# Get all image paths and labels
all_images = dataset.image_paths
all_labels = dataset.labels

# Split into 80% training, 20% testing
train_imgs, test_imgs, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42
)

print(f"Train Size: {len(train_imgs)}, Test Size: {len(test_imgs)}")


Train Size: 202, Test Size: 51


In [ ]:
test_dataset = BrainTumorDataset(DATASET_PATH, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(model, test_loader):
    model.eval()  # Set to evaluation mode
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Model Test Accuracy: {accuracy * 100:.2f}%")

evaluate(model, test_dataloader)


✅ Model Test Accuracy: 63.24%


In [ ]:
# Save model
torch.save(model.state_dict(), 'brain_tumor_model.pth')
print(" Model saved.")
